# Метод штрафных функций с правилом скорейшего спуска
###  Студент: Ефремова Ольга Игоревна 411 гр.

In [1]:
!pip install numdifftools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 1.3 MB/s 


In [2]:
import numpy as np
import pandas as pd
import numdifftools as nd
import scipy.optimize as sc

##Скорейший спуск
Пусть задача оптимизации имеет вид: $ F(x) \rightarrow inf ; x \in X, $\
Основная идея метода заключается в том, чтобы идти в направлении наискорейшего спуска, а это направление задаётся антиградиентом $-\nabla F$:\
 $x_{k+1} = x_k - \alpha_k \nabla F(x)$, где $\alpha_k$ задает скорость градиентного спуска.\
Для поиска минимума:
$F_k(\alpha) = F(x_k - \alpha \nabla F(x_k)) \rightarrow inf$ \
Используем $\alpha_k = argminF_k(\alpha)$\

###Алгоритм
1. Задают начальное приближение и точность расчёта $x_0$, $eps$
2. Рассчитывают $x_{k+1} = x_k - \alpha_k \nabla F(x)$, где $\alpha_k = argminF_k(\alpha)$
3. Проверяют условие остановки: $|| x_{k+1} - x_{k} ||< eps$ или $\nabla F(x_k) < eps$
4. Если останов не произошел, переходят к следующей итерации.

In [3]:
def mininize(f,eps, x_old):
  leng = (len(x_old))
  grad = nd.Gradient(f)
  x_new = x_old
  k = 0
  while k < 100:
    if np.linalg.norm(grad(x_old)) < eps:
      x_new = x_old
      break
    fk = lambda lam: f(x_old - lam*grad(x_old))
    res = sc.minimize_scalar(fk)
    l = res.x
    x_new = x_old - l*grad(x_old)
    if np.linalg.norm(x_new - x_old) < eps:
      break
    k = k + 1
    x_old = x_new
  return x_new

##Метод штрафных функций 
$$ F(x) \rightarrow inf ; u \in U, $$
$$ X = \{x \in X_0: g_i(x) \leq 0, i = \overline{1,n}; h_i(x) = 0, i = \overline{1,m}\}$$
Сводим к задаче\
$$F_k(x) = F(x) + A_k*P(x) \rightarrow inf$$ где
$A_k \rightarrow +\infty, k\rightarrow\infty$ и $A_k>0$, для любого $k$\
$P(x) = \sum(max(g_i(x), 0)^p | i= \overline{1,n}) + \sum(|h_i(x)|^p | i=\overline{1,m}), p\geq1$\
Останов, когда $A_k*P(x) < eps$

In [20]:
def penalty_method(f, g, h, eps, x_0):
    A = lambda k: k*k*30
    P = lambda x: sum(max(gi(x), 0)**p for gi in g) + sum(abs(hi(x))**p for hi in h)
    k = 1
    p=2
    x_old = x_0
    data = pd.DataFrame({'точка': [ '--------------------------', x_old], 'значение функции': ['------------------', '%.5f' %f(x_old)]})

    while(True):
        Fk = lambda x: f(x) + A(k)*P(x)
        x_new = mininize(Fk, eps, x_old)
        new_row = {'точка': np.round(x_new, 4), 'значение функции':f(x_new)}
        data = data.append(new_row, ignore_index=True)
        if A(k)*P(x_new) < eps:
            break
        x_old = x_new
        k = k + 1
    print(data)
    print('\nТочка минимума: ', x_new)
    print('Значение функции: ','%.10f' %f(x_new))
    print('Число итераций: ', k)
    if (gi(x_new)<= 0 for gi in g) and (hi(x_new) == 0 for hi in h):
      print('Nice')
    return (x_new,f(x_new))
    



##Задача 1
$F=x_0 + x_1 \rightarrow inf$\
Ограничения:\
$g_1 =  x_0^2 + x_1^2 -1 \leq 0$\
Начальная точка: $x = (1.0,0.5)$

In [ ]:
f = lambda x: x[0]+x[1]
g1 = lambda x: x[0]**2 + x[1]**2 - 1
g = [g1]
h = []
eps = 0.001
x_0 = np.array((1.0, 0.5))

penalty_method(f, g, h, eps, x_0)

                        точка    значение функции
0  --------------------------  ------------------
1                  [1.0, 0.5]              1.5000
2          [-0.7414, -0.6811]           -1.422458
3          [-0.7201, -0.6969]           -1.417004
4          [-0.7191, -0.6959]           -1.414948

Точка минимума:  [-0.7190846  -0.69586321]
Значение функции:  -1.4149478080338218
Число итераций:  3
Nice


##Задача 2
$F=x_0^2 + x_1^2 + 2x_2^2 + x_3^2 - 5x_0 - 5x_1 - 21x_2 + 7x_3 \rightarrow inf$\
Ограничения:\
$g_1 =  -(8 - x_0^2 - x_1^2 - x_2^2 - x_3^2 - x_0 + x_1 - x_2 + x_3)\leq 0$\
$g_2 = -(10 - x_0^2 - 2x_1^2 - x_2^2 - 2x_3^2 + x_0 - x_3)\leq 0$\
$g_3 =-(5 - 2x_0^2 - x_1^2 - x_2^2 - 2x_0 + x_1 + x_3)\leq 0$\
Начальная точка: $x = (0.0,0.0,0.0,0.0)$

In [21]:
f  = lambda x: x[0]**2 + x[1]**2 + 2*x[2]**2 + x[3]**2 - 5*x[0] - 5*x[1] - 21*x[2] + 7*x[3]
# Ограничения типа неравенства
g1 = lambda x: -(8 - x[0]**2 - x[1]**2 - x[2]**2 - x[3]**2 - x[0] + x[1] - x[2] + x[3])
g2 = lambda x: -(10 - x[0]**2 - 2*x[1]**2 - x[2]**2 - 2*x[3]**2 + x[0] - x[3])
g3 = lambda x: -(5 - 2*x[0]**2 - x[1]**2 - x[2]**2 - 2*x[0] + x[1] + x[3])
g = [g1, g2, g3]
# Ограничения типа равенства (их нет)
h = []

eps = 0.001
x_0 = np.array((0.0, 0.0, 0.0, 0.0))

penalty_method(f, g, h, eps, x_0)

                                точка    значение функции
0          --------------------------  ------------------
1                [0.0, 0.0, 0.0, 0.0]             0.00000
2   [0.0056, 0.9433, 2.0228, -0.9834]          -44.066344
3   [-0.001, 0.9436, 2.0187, -0.9883]          -44.006368
4   [-0.0012, 0.9435, 2.0181, -0.988]          -43.996275
5   [-0.0014, 0.9435, 2.0178, -0.988]          -43.990404
6   [-0.0016, 0.9434, 2.0176, -0.988]          -43.986615
7   [-0.0017, 0.9434, 2.0174, -0.988]          -43.984738
8   [-0.0017, 0.9434, 2.0174, -0.988]           -43.98362
9  [-0.0022, 0.9433, 2.0174, -0.9883]          -43.982757

Точка минимума:  [-0.00224786  0.94333174  2.01742908 -0.98827692]
Значение функции:  -43.9827571365
Число итераций:  8
Nice


(array([-0.00224786,  0.94333174,  2.01742908, -0.98827692]),
 -43.982757136454005)

##Задача 3
$F=150 \sum\limits_{i=2}^6{(x_i - ix_1)^4} + (x_1 - 2)^2 \rightarrow inf$  
Ограничения:\
$g_1 = \sum\limits_{i=1}^6{x_i^2} - 363 \leq 0$\
Начальные точи:\
$x_1 = (1.0, 2.0, 3.0, 4.0, 5.0, 6.0)$\
$x_1 = (2.0, 4.0, 6.0, 8.0, 10.0, 12.0)$

In [22]:
f  = lambda x: 150*(sum((x[i] - (i+1)*x[0])**4 for i in range(1,6))) + (x[0] - 2)**2
# Ограничения типа неравенства
g1 = lambda x: sum((x[i])**2 for i in range(0,6)) - 363
g = [g1]
# Ограничения типа равенства (их нет)
h = []

eps = 0.001
x_0 = np.array((2.0, 2.0, 2.0, 2.0, 2.0, 2.0))
x_1 = np.array((2.0, 4.0, 6.0, 8.0, 10.0, 12.0))
penalty_method(f, g, h, eps, x_0)
penalty_method(f, g, h, eps, x_1)

                                              точка    значение функции
0                        --------------------------  ------------------
1                    [2.0, 2.0, 2.0, 2.0, 2.0, 2.0]       2349600.00000
2  [0.7622, 1.4712, 2.2228, 2.9758, 3.7295, 4.4837]            1.556489

Точка минимума:  [0.76224375 1.47122858 2.22284013 2.97575992 3.72946912 4.48371332]
Значение функции:  1.5564887068
Число итераций:  1
Nice
                                               точка    значение функции
0                         --------------------------  ------------------
1                   [2.0, 4.0, 6.0, 8.0, 10.0, 12.0]             0.00000
2  [1.9985, 3.9945, 5.9917, 7.9889, 9.9862, 11.9834]            0.000003

Точка минимума:  [ 1.9985218   3.99446853  5.99170697  7.98894586  9.98618471 11.98342349]
Значение функции:  0.0000031145
Число итераций:  1
Nice


(array([ 1.9985218 ,  3.99446853,  5.99170697,  7.98894586,  9.98618471,
        11.98342349]), 3.114526338523022e-06)

##Задача 4
$F=150 \sum\limits_{i=2}^6{(x_i - ix_1)^4} + (x_1 - 2)^2 \rightarrow inf$  
Ограничения:\
$g_1 = \sum\limits_{i=1}^6{x_i^2} - 363 \leq 0$\
Начальные точи:\
$(x,x,x,x,x,x)$, где $x \in (-50,50)$

In [25]:
f  = lambda x: 150*(sum((x[i] - (i+1)*x[0])**4 for i in range(1,6))) + (x[0] - 2)**2
# Ограничения типа неравенства
g1 = lambda x: sum((x[i])**2 for i in range(0,6)) - 363
g = [g1]
# Ограничения типа равенства (их нет)
h = []
data1 = pd.DataFrame({'начальная точка': [], 'точка минимума':[], 'значение функции': []})

eps = 0.001
x = -50
while x <= 50:
  x_1 = [x,x,x,x,x,x]
  x_new, res = penalty_method(f, g, h, eps, x_1)
  new_row = {'начальная точка': x_1, 'точка минимума': x_new, 'значение функции': '%.7f' %res}
  data1 = data1.append(new_row, ignore_index=True)
  x+=1
data1.to_csv('./result.csv')

                                               точка    значение функции
0                         --------------------------  ------------------
1                     [-50, -50, -50, -50, -50, -50]  917812502704.00000
2  [-1.7808, -3.6341, -5.4253, -7.2151, -9.0021, ...            14.35006

Точка минимума:  [ -1.78075851  -3.63407872  -5.42528437  -7.2151439   -9.0020672
 -10.79181419]
Значение функции:  14.3500596910
Число итераций:  1
Nice
                                               точка    значение функции
0                         --------------------------  ------------------
1                     [-49, -49, -49, -49, -49, -49]  846561029451.00000
2  [-1.8, -3.6727, -5.483, -7.292, -9.0979, -10.9...           14.495442

Точка минимума:  [ -1.8000065   -3.67267008  -5.48302593  -7.29203475  -9.09788823
 -10.90693762]
Значение функции:  14.4954423649
Число итераций:  1
Nice
                                               точка    значение функции
0                         ------